# **`Project 3: Team Mary Paley`**

### **Objective**: ...

#### Group Members:
> Emily Wu: wuemily@berkeley.edu <br>
> Rania Nasser: ...@berkeley.edu <br>
> Jing Huang: ...@berkeley.edu <br>
> Carmen Vega: carmenvega@berkeley.edu <br>
> Mario Zhao: mario.zhao@berkeley.edu <br>

### **[A]: Choice of a population, with supporting expenditure data**

In [7]:
apikey = "KNqUDtV7Kcktiuheo3EoNhB0zDlCevFAdqZrKgdj"
%pip install -r requirements.txt --upgrade
%pip install wbdata
import fooddatacentral as fdc
import pandas as pd
import numpy as np
import re
from  scipy.optimize import linprog as lp
import wbdata
import sys
import io
import warnings
warnings.simplefilter("ignore")

  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Ethiopia Data**

In [3]:
eth_hc = pd.read_csv('Data Files/Ethiopia - Household Characteristics.csv')
eth_hc = eth_hc[eth_hc['t'] == '2015-16']
eth_hc.head()

,i,t,m,Females 00-03,Females 04-08,Females 09-13,Females 14-18,Females 19-30,Females 31-50,Females 51-99,Males 00-03,Males 04-08,Males 09-13,Males 14-18,Males 19-30,Males 31-50,Males 51-99,log HSize
2,1010101601002,2015-16,Tigray,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000
5,1010101601017,2015-16,Tigray,1,1,2,0,0,1,0,0,1,0,2,0,1,0,2.197225
8,1010101601034,2015-16,Tigray,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.000000
11,1010101601049,2015-16,Tigray,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1.098612
14,1010101601064,2015-16,Tigray,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.693147


In [4]:
eth_fe = pd.read_csv('Data Files/Ethiopia - Food Expenditures (2015-16).csv')
eth_fe.head()

,i,t,m,j,Expenditure
0,1010101601002,2015-16,Tigray,Horsebeans,15.0
1,1010101601002,2015-16,Tigray,Oils,4.0
2,1010101601002,2015-16,Tigray,Onion,6.0
3,1010101601002,2015-16,Tigray,Tomato,7.0
4,1010101601017,2015-16,Tigray,Eggs,12.0


In [5]:
eth_fp = pd.read_csv('Data Files/Ethiopia - Food Prices (2015-16).csv')
eth_fp.head()

,t,m,j,u,Price
0,2015-16,Addis Ababa,Banana,Kg,15.826637
1,2015-16,Addis Ababa,Barley,Kg,24.614171
2,2015-16,Addis Ababa,Beef,Kg,1722.234772
3,2015-16,Addis Ababa,Beer,Kg,2155.527200
4,2015-16,Addis Ababa,Berbere,Kg,182.433182


**Ghana Data**

In [11]:
usa_hc = pd.read_csv('Data Files/WWEIA - Household Characteristics 2015.csv')
usa_hc = usa_hc[usa_hc['t'] == 2015]
usa_hc.head()

,i,t,m,age,sex,HH_size
0,83732,2015,USA,62.0,Male,2
1,83733,2015,USA,53.0,Male,1
2,83734,2015,USA,78.0,Male,2
3,83735,2015,USA,56.0,Female,1
4,83736,2015,USA,42.0,Female,5


In [12]:
usa_fe = pd.read_csv('Data Files/WWEIA - Food Expenditures 2015.csv')
usa_fe.head()

,i,j,t,m,grams,food_name
0,83732,11111000,2015,USA,488.00,"Milk, whole"
1,83732,12210400,2015,USA,2.94,"Coffee creamer, powder"
2,83732,21500310,2015,USA,113.33,"Ground beef patty, cooked"
3,83732,22311020,2015,USA,126.00,"Ham, smoked or cured, cooked, lean only eaten"
4,83732,24127500,2015,USA,175.00,"Chicken breast, baked, coated, skin / coating ..."


In [13]:
#usa_fp = ...
#usa_fp.head()

### **[B]: Nutritional content of different foods**

### **[B]: Nutritional adequacy of diet**

### **[A]: Estimate demand system**

### **[C]: Counterfactual experiments**